In [18]:
from census import Census
from config import (census1_api_key, census2_api_key, weather_api_key, gkey)
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import os
import numpy as np
from census import Census
import gmaps
import csv
import matplotlib.pyplot as plt
import openweathermapy as ow
import time


# Geo Maps

In [19]:
county_df = pd.read_csv("county_coordinates_edits.csv").iloc[:,1:5]
county_df["County"] = county_df["County"].map(lambda x: x.lower())
county_df["State"] = county_df["State"].map(lambda x: x.lower())
county_df["State"] = county_df['State'].str.strip()
combined_data = pd.read_csv("combined_data.csv").iloc[:,1:8]

In [20]:
geo_plot_data=pd.merge(combined_data,county_df,on=["County","State"],how="outer")


In [21]:
geo_plot_data["Homeless Rate"] = geo_plot_data["Homeless Pop"] /geo_plot_data["Population"]*100


In [22]:
geo_plot_data=geo_plot_data.loc[geo_plot_data["Homeless Rate"]>0,:]
geo_plot_data.head()

,County,State,Homeless Pop,Population,Poverty Rate,Price_2018,Temp,lat,lng,Homeless Rate
4,coffee county,alabama,64.0,51073.0,15.902727,109500.000045,41.00,35.46,-86.05,0.125311
7,etowah county,alabama,52.0,103132.0,17.614320,91299.999998,57.00,34.05,-86.05,0.050421
9,lee county,alabama,103.0,156597.0,21.157493,178599.999979,42.01,26.66,-81.95,0.065774
10,houston county,alabama,103.0,104108.0,18.279095,128400.000043,30.20,32.42,-83.63,0.098936
11,jefferson county,alabama,1958.0,659460.0,17.191035,109000.000012,45.00,39.58,-105.27,0.296910


In [23]:
geo_plot_data=geo_plot_data.dropna()
geo_plot_data.describe()

,Homeless Pop,Population,Poverty Rate,Price_2018,Temp,lat,lng,Homeless Rate
count,969.000000,9.690000e+02,969.000000,9.690000e+02,969.000000,969.000000,969.000000,969.000000
mean,704.083591,2.697216e+05,14.315934,1.944044e+05,48.049845,38.508958,-91.468060,0.218780
std,2126.511557,5.492992e+05,4.984613,1.327459e+05,15.936229,5.176648,15.716979,0.210805
min,3.000000,6.382000e+03,2.925666,2.150000e+04,14.590000,19.540000,-159.570000,0.004106
25%,61.000000,5.190600e+04,10.833990,1.179000e+05,35.010000,34.960000,-97.700000,0.083432
50%,163.000000,1.076990e+05,13.942620,1.591000e+05,43.000000,39.240000,-85.520000,0.169559
75%,522.000000,2.673900e+05,17.093008,2.258000e+05,64.400000,42.000000,-80.720000,0.286807
max,31673.000000,1.010572e+07,37.181018,1.310900e+06,79.000000,64.950000,-68.580000,2.232555


In [24]:
# Configure gmaps
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations 
locations = geo_plot_data[["lat", "lng"]].astype(float)

# Convert Poverty Rate to float and store
poverty_rate = geo_plot_data["Poverty Rate"].astype(float)

# Create a poverty Heatmap layer


p_heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, dissipating=False, max_intensity=100,point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
p_heat_layer.dissipating = False
p_heat_layer.max_intensity = 100
p_heat_layer.point_radius = 1

In [25]:
homeless_rate=(geo_plot_data["Homeless Rate"]/.15+1).astype("int64").tolist()
house_prices=(geo_plot_data["Price_2018"]/160000+1).astype("int64").tolist()
temp=(geo_plot_data["Temp"]/43+1).astype("int64").tolist()

In [26]:
# Convert Poverty Rate to float and store
#homeless_count = merge_table_all["Homeless Pop"].tolist()
homeless_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 0, 0, 0)',
    stroke_color='rgba(0, 0, 0, .4)', scale=homeless_rate)

house_prices_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 0, 0, 0)',
    stroke_color='rgba(0, 0, 0, 0.4)', scale=house_prices)

temp_layer=gmaps.symbol_layer(
    locations, fill_color='rgba(0,0,0,0)',
    stroke_color='rgba(0,0,0,.4)',scale=temp)

# Poverty/Homelessness GeoMap

In [28]:
fig = gmaps.figure()
fig.add_layer(p_heat_layer)
fig.add_layer(homeless_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# House Prices/Homelessness GeoMap

In [29]:
fig = gmaps.figure()
fig.add_layer(p_heat_layer)
fig.add_layer(house_prices_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Temperature/Homelessness GeoMap

In [30]:
fig = gmaps.figure()
fig.add_layer(p_heat_layer)
fig.add_layer(temp_layer)
fig

Figure(layout=FigureLayout(height='420px'))